In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
# Importing data
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [3]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
# Preparing the training and test sets
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')

In [7]:
training_set.head()

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561


In [8]:
test_set.head()

,1,6,5,887431973
0,1,10,3,875693118
1,1,12,5,878542960
2,1,14,5,874965706
3,1,17,3,875073198
4,1,20,4,887431883


In [9]:
training_set = np.array(training_set, dtype = int)
test_set = np.array(test_set, dtype = int)

In [10]:
# Getting the number of users and movies
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))

In [11]:
# convert data into an array, list of list, users in lines and movies in columns
def convert(data):
    new_data =[]
    for id_users in range(1, nb_users + 1):
        id_movies = data[:, 1][data[:, 0] == id_users]
        id_ratings = data[:, 2][data[:, 0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [12]:
training_set_convert = convert(training_set)
test_set_convert = convert(test_set)

In [13]:
# Convert data into Torch tensor
training_set_convert_tensor = torch.FloatTensor(training_set_convert)
test_set_convert_tensor = torch.FloatTensor(test_set_convert)

In [14]:
# Creat the architecture of neural network
"""
This class is an inheritance class
it means a child class from a parent class
"""  
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20) # 20 is the number of hidden nodes in 1st layer, you can optimize it
                                            # in fact these are 20 features in 1st hidden layer
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()      
criterian = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [15]:
# Training Stack Auto Encoder
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set_convert_tensor[id_user]).unsqueeze(0) # torch does not accept single vector
                                                                            # so we need to generate a batch of vectors
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False # we don't compute gradient respect to the target
            output[target == 0] = 0
            loss = criterian(output, target)
            mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10) # nb of movies with non zero ratings
            loss.backward() # it says the loss is increasing or decreasing, or direction of loss
            train_loss += np.sqrt(loss.data * mean_corrector)
            s += 1.
            optimizer.step() # says the intensity of weights
    print('epoch: ' + str(epoch) + ' loss: ' + str(train_loss / s))

epoch: 1 loss: tensor(1.7712)
epoch: 2 loss: tensor(1.0967)
epoch: 3 loss: tensor(1.0533)
epoch: 4 loss: tensor(1.0385)
epoch: 5 loss: tensor(1.0306)
epoch: 6 loss: tensor(1.0266)
epoch: 7 loss: tensor(1.0237)
epoch: 8 loss: tensor(1.0220)
epoch: 9 loss: tensor(1.0206)
epoch: 10 loss: tensor(1.0196)
epoch: 11 loss: tensor(1.0187)
epoch: 12 loss: tensor(1.0183)
epoch: 13 loss: tensor(1.0179)
epoch: 14 loss: tensor(1.0175)
epoch: 15 loss: tensor(1.0171)
epoch: 16 loss: tensor(1.0171)
epoch: 17 loss: tensor(1.0166)
epoch: 18 loss: tensor(1.0165)
epoch: 19 loss: tensor(1.0163)
epoch: 20 loss: tensor(1.0163)
epoch: 21 loss: tensor(1.0160)
epoch: 22 loss: tensor(1.0160)
epoch: 23 loss: tensor(1.0158)
epoch: 24 loss: tensor(1.0156)
epoch: 25 loss: tensor(1.0155)
epoch: 26 loss: tensor(1.0155)
epoch: 27 loss: tensor(1.0153)
epoch: 28 loss: tensor(1.0150)
epoch: 29 loss: tensor(1.0127)
epoch: 30 loss: tensor(1.0114)
epoch: 31 loss: tensor(1.0100)
epoch: 32 loss: tensor(1.0083)
epoch: 33 loss: t

In [23]:
# Testing SAE
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set_convert_tensor[id_user]).unsqueeze(0)
    target = Variable(test_set_convert_tensor[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterian(output, target)
        mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.item() * mean_corrector) # this line is modifies
        s += 1.
print('test loss: ' + str(test_loss / s))

test loss: 0.9493352340444207
